In [1]:
import tensorflow as tf
print(tf.__version__)

2025-06-19 12:19:26.666648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750335566.937850      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750335567.021656      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.18.0


In [2]:
import pandas as pd
import numpy as np

csv_path = "/kaggle/input/seminar-lstm/Gesamterzeugung_hourly.csv"
neu = pd.read_csv(csv_path, index_col='Datum', parse_dates=True)
print(f"🕐 Typ der ersten Spalte: {type(neu.iloc[1, 0])}")
display(neu.iloc[50000:50003])
neu.head(-3)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/seminar-lstm/Gesamterzeugung_hourly.csv'

In [ ]:
import tensorflow as tf
seq_length = 336 #(336)

#Preprocessing, Normalisierung


Strom_train = neu["Stromerzeugung Gesamt"]["2015-01-01 00:00:00":"2018-06-30 23:00:00"]
mean = Strom_train.mean()
std = Strom_train.std()

#Dataset in Training Validation Testset einteilen

Strom_train = (((neu["Stromerzeugung Gesamt"]["2015-01-01 00:00:00":"2018-06-30 23:00:00"]) - mean)/std)

Strom_valid = (((neu["Stromerzeugung Gesamt"]["2018-07-01 00:00:00":"2018-12-31 23:00:00"]) - mean)/std)

Strom_test = (((neu["Stromerzeugung Gesamt"]["2019-01-01 00:00:00":"2019-06-01 23:00:00"]) - mean)/std)


#Training und Validation Set aufteilen in Targets & Inputs

def split_inputs_and_targets(time_series, ahead=24): # target_col=1):
 return time_series[:, :-ahead], time_series[:, -ahead:]

train_ds = tf.keras.utils.timeseries_dataset_from_array(
 Strom_train.to_numpy(),
 targets=None,
 sequence_length=seq_length + 24,
 batch_size=128,
 shuffle=False,
 seed=42
).map(split_inputs_and_targets)

valid_ds = tf.keras.utils.timeseries_dataset_from_array(
 Strom_valid.to_numpy(),
 targets=None,
 sequence_length=seq_length + 24,
 batch_size=128
).map(split_inputs_and_targets)



print("train_ds shape:")
inputs, targets = next(iter(train_ds))
print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)
print("valid_ds shape:")
inputs, targets = next(iter(valid_ds))
print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)


In [ ]:
import os
import pickle
import tensorflow as tf
from IPython.display import FileLink  # Für Kaggle-Downloads

tf.random.set_seed(42)


# Modellname und Basis-Speicherpfad (Kaggle-kompatibel)
model_name = "model_GRU32x2_gradclip_RedPl_seq336_B128"
base_model_dir = "./models/"  # 🆕 Lokaler Pfad in Kaggle


# Unterordner für das Modell erstellen
model_dir = os.path.join(base_model_dir, model_name)
os.makedirs(model_dir, exist_ok=True)

# Pfade im Unterordner
model_path = os.path.join(model_dir, f"{model_name}-epoch-{{epoch:02d}}.keras")
history_path = os.path.join(model_dir, f"{model_name}_history.pkl")


model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 1)),
    tf.keras.layers.GRU(32, 
                        dropout=0.1,
                        recurrent_dropout=0.2,
                        return_sequences=True),
    tf.keras.layers.GRU(32, dropout=0.1),
    tf.keras.layers.Dense(24)
])


opt = tf.keras.optimizers.Adam(
    learning_rate=0.005,
    clipvalue=1.0
)




lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_best_only=False,
    save_freq="epoch",
    verbose=1
)

model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=["mae"]
)

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=100,
    callbacks=[model_checkpoint_cb, lr_scheduler]
)

# History speichern
with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)

print(f"\nGespeicherte Dateien in Kaggle:")
print(f"- Modell-Checkpoints: {model_dir}/*.keras")
print(f"- Trainingshistorie: {history_path}")

#Download-Links für Kaggle
# Erstelle ZIP für einfachen Download
import shutil
shutil.make_archive(model_name, 'zip', model_dir)
display(FileLink(f"{model_name}.zip"))  # Download-Link für gesamtes Modell

In [ ]:
import os
import pickle
import tensorflow as tf
from IPython.display import FileLink  # Für Kaggle-Downloads

tf.random.set_seed(42)


# Modellname und Basis-Speicherpfad (Kaggle-kompatibel)
model_name = "model_GRU32x2_gradclip_RedPl_seq336_B128"
base_model_dir = "./models/"  # 🆕 Lokaler Pfad in Kaggle


# Unterordner für das Modell erstellen
model_dir = os.path.join(base_model_dir, model_name)
os.makedirs(model_dir, exist_ok=True)

# Pfade im Unterordner
model_path = os.path.join(model_dir, f"{model_name}-epoch-{{epoch:02d}}.keras")
history_path = os.path.join(model_dir, f"{model_name}_history.pkl")


model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 1)),
    tf.keras.layers.GRU(32, 
                        dropout=0.1,
                        recurrent_dropout=0.2,
                        return_sequences=True),
    tf.keras.layers.GRU(32, dropout=0.1),
    tf.keras.layers.Dense(24)
])


opt = tf.keras.optimizers.Adam(
    learning_rate=0.005,
    clipvalue=1.0
)




lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_best_only=False,
    save_freq="epoch",
    verbose=1
)

model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=["mae"]
)

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=100,
    callbacks=[model_checkpoint_cb, lr_scheduler]
)

# History speichern
with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)

print(f"\nGespeicherte Dateien in Kaggle:")
print(f"- Modell-Checkpoints: {model_dir}/*.keras")
print(f"- Trainingshistorie: {history_path}")

#Download-Links für Kaggle
# Erstelle ZIP für einfachen Download
import shutil
shutil.make_archive(model_name, 'zip', model_dir)
display(FileLink(f"{model_name}.zip"))  # Download-Link für gesamtes Modell

In [ ]:
print 

In [ ]:
import tensorflow as tf
seq_length = 336 #(336)

#Preprocessing, Normalisierung


Strom_train = neu["Stromerzeugung Gesamt"]["2015-01-01 00:00:00":"2018-06-30 23:00:00"]
mean = Strom_train.mean()
std = Strom_train.std()

#Dataset in Training Validation Testset einteilen

Strom_train = (((neu["Stromerzeugung Gesamt"]["2015-01-01 00:00:00":"2018-06-30 23:00:00"]) - mean)/std)

Strom_valid = (((neu["Stromerzeugung Gesamt"]["2018-07-01 00:00:00":"2018-12-31 23:00:00"]) - mean)/std)

Strom_test = (((neu["Stromerzeugung Gesamt"]["2019-01-01 00:00:00":"2019-06-01 23:00:00"]) - mean)/std)


#Training und Validation Set aufteilen in Targets & Inputs

def split_inputs_and_targets1(time_series, ahead=1): # target_col=1):
 return time_series[:, :-ahead], time_series[:, -ahead:]
    
train_ds1 = tf.keras.utils.timeseries_dataset_from_array(
    Strom_train.to_numpy(),
    targets=None,
    sequence_length=seq_length + 1,  
    batch_size=128,
    shuffle=False,
    seed=42
).map(split_inputs_and_targets1)

valid_ds1 = tf.keras.utils.timeseries_dataset_from_array(
    Strom_valid.to_numpy(),
    targets=None,
    sequence_length=seq_length + 1,
    batch_size=128
).map(split_inputs_and_targets1)



print("train_ds1 shape:")
inputs, targets = next(iter(train_ds1))
print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)
print("valid_ds1 shape:")
inputs, targets = next(iter(valid_ds1))
print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)

In [ ]:
import os
import pickle
import tensorflow as tf
from IPython.display import FileLink  # Für Kaggle-Downloads

tf.random.set_seed(42)


# Modellname und Basis-Speicherpfad (Kaggle-kompatibel)
model_name = "model_1Hour_GRU32x2_gradclip_RedPl_seq336_B128"
base_model_dir = "./models/"  # 🆕 Lokaler Pfad in Kaggle


# Unterordner für das Modell erstellen
model_dir = os.path.join(base_model_dir, model_name)
os.makedirs(model_dir, exist_ok=True)

# Pfade im Unterordner
model_path = os.path.join(model_dir, f"{model_name}-epoch-{{epoch:02d}}.keras")
history_path = os.path.join(model_dir, f"{model_name}_history.pkl")


model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 1)),
    tf.keras.layers.GRU(32, 
                        dropout=0.1,
                        recurrent_dropout=0.2,
                        return_sequences=True),
    tf.keras.layers.GRU(32, dropout=0.1),
    tf.keras.layers.Dense(1)
])


opt = tf.keras.optimizers.Adam(
    learning_rate=0.005,
    clipvalue=1.0
)




lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_best_only=False,
    save_freq="epoch",
    verbose=1
)

model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=["mae"]
)

history = model.fit(
    train_ds1,
    validation_data=valid_ds1,
    epochs=100,
    callbacks=[model_checkpoint_cb, lr_scheduler]
)

# History speichern
with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)

print(f"\nGespeicherte Dateien in Kaggle:")
print(f"- Modell-Checkpoints: {model_dir}/*.keras")
print(f"- Trainingshistorie: {history_path}")

#Download-Links für Kaggle
# Erstelle ZIP für einfachen Download
import shutil
shutil.make_archive(model_name, 'zip', model_dir)
display(FileLink(f"{model_name}.zip"))  # Download-Link für gesamtes Modell

In [ ]:
import os
import pickle
import tensorflow as tf
from IPython.display import FileLink  # Für Kaggle-Downloads

tf.random.set_seed(42)


# Modellname und Basis-Speicherpfad (Kaggle-kompatibel) 
model_name = "model_1Hour_LSTM32x2_gradclip_RedPl_seq336_B128"
base_model_dir = "./models/"  # 🆕 Lokaler Pfad in Kaggle


# Unterordner für das Modell erstellen
model_dir = os.path.join(base_model_dir, model_name)
os.makedirs(model_dir, exist_ok=True)

# Pfade im Unterordner
model_path = os.path.join(model_dir, f"{model_name}-epoch-{{epoch:02d}}.keras")
history_path = os.path.join(model_dir, f"{model_name}_history.pkl")


model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 1)),
    tf.keras.layers.LSTM(32, 
                        dropout=0.1,
                        recurrent_dropout=0.2,
                        return_sequences=True),
    tf.keras.layers.LSTM(32, dropout=0.1),
    tf.keras.layers.Dense(1)
])


opt = tf.keras.optimizers.Adam(
    learning_rate=0.005,
    clipvalue=1.0
)




lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_best_only=False,
    save_freq="epoch",
    verbose=1
)

model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=["mae"]
)

history = model.fit(
    train_ds1,
    validation_data=valid_ds1,
    epochs=100,
    callbacks=[model_checkpoint_cb, lr_scheduler]
)

# History speichern
with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)

print(f"\nGespeicherte Dateien in Kaggle:")
print(f"- Modell-Checkpoints: {model_dir}/*.keras")
print(f"- Trainingshistorie: {history_path}")

#Download-Links für Kaggle
# Erstelle ZIP für einfachen Download
import shutil
shutil.make_archive(model_name, 'zip', model_dir)
display(FileLink(f"{model_name}.zip"))  # Download-Link für gesamtes Modell

In [ ]:
import tensorflow as tf
seq_length = 336 #(336)

#Preprocessing, Normalisierung


Strom_train = neu["Stromerzeugung Gesamt"]["2015-01-01 00:00:00":"2018-06-30 23:00:00"]
mean = Strom_train.mean()
std = Strom_train.std()

#Dataset in Training Validation Testset einteilen

Strom_train = (((neu["Stromerzeugung Gesamt"]["2015-01-01 00:00:00":"2018-06-30 23:00:00"]) - mean)/std)

Strom_valid = (((neu["Stromerzeugung Gesamt"]["2018-07-01 00:00:00":"2018-12-31 23:00:00"]) - mean)/std)

Strom_test = (((neu["Stromerzeugung Gesamt"]["2019-01-01 00:00:00":"2019-06-01 23:00:00"]) - mean)/std)


#Training und Validation Set aufteilen in Targets & Inputs

def split_inputs_and_targets1(time_series, ahead=1): # target_col=1):
 return time_series[:, :-ahead], time_series[:, -ahead:]
    
train_ds1 = tf.keras.utils.timeseries_dataset_from_array(
    Strom_train.to_numpy(),
    targets=None,
    sequence_length=seq_length + 1,  
    batch_size=64,
    shuffle=False,
    seed=42
).map(split_inputs_and_targets1)

valid_ds1 = tf.keras.utils.timeseries_dataset_from_array(
    Strom_valid.to_numpy(),
    targets=None,
    sequence_length=seq_length + 1,
    batch_size=64
).map(split_inputs_and_targets1)



print("train_ds1 shape:")
inputs, targets = next(iter(train_ds1))
print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)
print("valid_ds1 shape:")
inputs, targets = next(iter(valid_ds1))
print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)

In [ ]:
import os
import pickle
import tensorflow as tf
from IPython.display import FileLink  # Für Kaggle-Downloads

tf.random.set_seed(42)


# Modellname und Basis-Speicherpfad (Kaggle-kompatibel)
model_name = "model_1Hour_LSTM64_gradclip_RedPl_seq336"
base_model_dir = "./models/"  # 🆕 Lokaler Pfad in Kaggle


# Unterordner für das Modell erstellen
model_dir = os.path.join(base_model_dir, model_name)
os.makedirs(model_dir, exist_ok=True)

# Pfade im Unterordner
model_path = os.path.join(model_dir, f"{model_name}-epoch-{{epoch:02d}}.keras")
history_path = os.path.join(model_dir, f"{model_name}_history.pkl")


model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 1)),    
    tf.keras.layers.LSTM(64, dropout=0.1),
    tf.keras.layers.Dense(1)
])


opt = tf.keras.optimizers.Adam(
    learning_rate=0.005,
    clipvalue=1.0
)




lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_best_only=False,
    save_freq="epoch",
    verbose=1
)

model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=["mae"]
)

history = model.fit(
    train_ds1,
    validation_data=valid_ds1,
    epochs=100,
    callbacks=[model_checkpoint_cb, lr_scheduler]
)

# History speichern
with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)

print(f"\nGespeicherte Dateien in Kaggle:")
print(f"- Modell-Checkpoints: {model_dir}/*.keras")
print(f"- Trainingshistorie: {history_path}")

#Download-Links für Kaggle
# Erstelle ZIP für einfachen Download
import shutil
shutil.make_archive(model_name, 'zip', model_dir)
display(FileLink(f"{model_name}.zip"))  # Download-Link für gesamtes Modell

In [ ]:
import os
import pickle
import tensorflow as tf
from IPython.display import FileLink  # Für Kaggle-Downloads

tf.random.set_seed(42)


# Modellname und Basis-Speicherpfad (Kaggle-kompatibel)
model_name = "model_1Hour_GRU64_gradclip_RedPl_seq336"
base_model_dir = "./models/"  # 🆕 Lokaler Pfad in Kaggle


# Unterordner für das Modell erstellen
model_dir = os.path.join(base_model_dir, model_name)
os.makedirs(model_dir, exist_ok=True)

# Pfade im Unterordner
model_path = os.path.join(model_dir, f"{model_name}-epoch-{{epoch:02d}}.keras")
history_path = os.path.join(model_dir, f"{model_name}_history.pkl")


model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 1)),    
    tf.keras.layers.GRU(64, dropout=0.1),
    tf.keras.layers.Dense(1)
])


opt = tf.keras.optimizers.Adam(
    learning_rate=0.005,
    clipvalue=1.0
)




lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_best_only=False,
    save_freq="epoch",
    verbose=1
)

model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=["mae"]
)

history = model.fit(
    train_ds1,
    validation_data=valid_ds1,
    epochs=100,
    callbacks=[model_checkpoint_cb, lr_scheduler]
)

# History speichern
with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)

print(f"\nGespeicherte Dateien in Kaggle:")
print(f"- Modell-Checkpoints: {model_dir}/*.keras")
print(f"- Trainingshistorie: {history_path}")

#Download-Links für Kaggle
# Erstelle ZIP für einfachen Download
import shutil
shutil.make_archive(model_name, 'zip', model_dir)
display(FileLink(f"{model_name}.zip"))  # Download-Link für gesamtes Modell